# CALCULATE SERVICE COVERAGE OF BUS NETWORK

In [ ]:
import yaml
import pandas as pd
import numpy as np
from lxml import etree

# Set up Plan and Schedule

In [ ]:
PLAN_PATH = "data\\real\\plans_scale0.375true.xml"
SCHEDULE_PATH = "data\\real\\transit_schedule.xml"

## head plan file

In [ ]:
import os
print("Thư mục hiện tại:", os.getcwd())
print("Danh sách file/thư mục tại đây:", os.listdir())

# Get home coordinate

In [ ]:
!powershell -Command "Get-Content 'data\real\plans_scale0.375true.xml' -TotalCount 20 | ForEach-Object { '{0:5}: {1}' -f $_.ReadCount, $_ }"

In [ ]:
!@REM !powershell -Command "Get-Content 'data\simple_scenario\plans.xml' -TotalCount 20 | ForEach-Object { '{0:5}: {1}' -f $_.ReadCount, $_ }"@REM 

### Class lưu dữ liệu home

In [ ]:
class HomeCoordinate:
    def __init__(self, person_id: str, x: float, y: float):
        self.person_id = person_id
        self.x = x
        self.y = y



In [ ]:
parser = etree.XMLParser(remove_blank_text=True)
tree = etree.parse(PLAN_PATH, parser)
root = tree.getroot()


### Lọc các tọa độ home của mỗi người

In [ ]:
# print(len(root.xpath('//population/person/plan[@selected="yes"]/act[@type="home"][1]')))

# xpath luôn trả ra list
# /tag (Ở đầu)	Tìm trong Root (Gốc) - Luôn bắt đầu tìm từ đỉnh cao nhất của file XML trar ra các list có tag tương ứng.
# //tag (Ở đầu): 	Tìm trong toàn bộ file - Tìm tất cả các node khớp với tên đó, bất kể nó nằm ở đâu.
# ./tag hoặc Không có xuyệt: Relative (Tương đối) - Chỉ tìm bên trong node bạn đang đứng.
# @color là tên attribute
# [@attr='value']	Lọc theo attribute - Chỉ chọn các node có attribute là value. Trả ra 1 list

home_coordinates: list[HomeCoordinate] = []
residence_count = len(root.xpath('//population/person'))
print("Residence count: ", residence_count)

for node in root.xpath('//population/person'):
    id = node.xpath('@id')[0]
    act = node.xpath(('./plan[@selected = "yes"]/act[@type="home"]'))
    x = act[0].xpath('@x')[0]
    y = act[0].xpath('@y')[0]
    home_coordinates.append(HomeCoordinate(id, float(x), float(y)))
    
print("Home location count: ", len(home_coordinates))
    

# Get Bus Stop Coordinate

In [ ]:
!powershell -Command "Get-Content 'data\real\transit_schedule.xml' -TotalCount 90 | ForEach-Object { '{0:5}: {1}' -f $_.ReadCount, $_ }"

In [ ]:
!@REM !powershell -Command "Get-Content 'data\simple_scenario\schedule.xml' -TotalCount 90 | ForEach-Object { '{0:5}: {1}' -f $_.ReadCount, $_ }"

### class lưu dữ liệu stop

In [ ]:
class StopCoordinate:
    def __init__(self, stop_id: str, x: float, y: float):
        self.stop_id = stop_id
        self.x = x
        self.y = y


### Lọc các bến trong tuyến có transportmode string 'bus' ( ở dạng hoa hay thường)

In [ ]:
parser = etree.XMLParser(remove_blank_text=True)
tree = etree.parse(SCHEDULE_PATH, parser)
root = tree.getroot()

full_stop_coordinate : set[StopCoordinate] = set()
bus_stop_coordinate : set[StopCoordinate] = set()

for node in root.xpath('//transitSchedule/transitStops/stopFacility'):
    id = node.xpath("@id")[0]
    x = node.xpath("@x")[0]
    y = node.xpath("@y")[0]
    print([id, x, y])
    full_stop_coordinate.add(StopCoordinate(id, float(x), float(y)))


### LOGIC: 1 tuyến được cho là bus khi mà transportMode có chứa hint_bus_route

In [ ]:
hint_bus_route = "pt"

print("----- Bus Stops -----")
for node in root.xpath('//transitSchedule/transitLine/transitRoute'):
    results = node.xpath('transportMode/text()')[0]
    if hint_bus_route in str(results).lower():
        stop_tag_list = node.xpath("./routeProfile/stop")
        for stop in stop_tag_list:
            stop_id = stop.xpath('@refId')[0]
            for full_stop in full_stop_coordinate:
                if stop_id == full_stop.stop_id:
                    print([stop_id, full_stop.x, full_stop.y])
                    bus_stop_coordinate.add(full_stop)
                    break


# Calculate Service Coverage

In [ ]:
def calculte_service_coverage(home_coor_list: list[HomeCoordinate], stop_coor_list: set[StopCoordinate], radia: float):
    service_coverage = 0
    for home_coor in home_coor_list:
        for stop_coor in stop_coor_list:
            distance = np.sqrt((home_coor.x - stop_coor.x)**2 + (home_coor.y - stop_coor.y)**2)
            if distance <= radia:
                service_coverage += 1
                break
    return service_coverage

service_coverage = calculte_service_coverage(home_coordinates, bus_stop_coordinate, 400)
print(f"Service coverage Bus Stops (400m): ${service_coverage};  percent: {service_coverage/residence_count*100:.2f}%")
